# 🎯 악성 판매자 탐지 모델 - 인터랙티브 분석

**마우스를 올리면 상세 정보가 나타납니다!**

## 📦 패키지 설치 및 Import

In [1]:
# 필요한 패키지 설치 (처음 한 번만)
# !pip install plotly kaleido

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("✅ 패키지 import 완료")

✅ 패키지 import 완료


## 📊 1. 데이터 로드

In [3]:
# 데이터 로드
df = pd.read_csv('../output/seller_features.csv')
feature_importance = pd.read_csv('../output/feature_importance.csv')
predictions = pd.read_csv('../output/prediction_results.csv')

print(f"✅ 데이터 크기: {df.shape}")
print(f"✅ Feature Importance: {len(feature_importance)}개")
print(f"✅ 예측 결과: {len(predictions)}개")

# 라벨 이름 매핑
df['label_name'] = df['abusive_label'].map({0: '정상', 1: '악성'})
df.head()

✅ 데이터 크기: (374, 23)
✅ Feature Importance: 20개
✅ 예측 결과: 75개


,vendor_name,avg_rating,rating_std,low_rating_ratio,avg_review_length,duplicate_review_ratio,negative_keyword_ratio,textless_5star_ratio,review_count,review_density,...,question_density,question_review_ratio,negative_sentiment_ratio,avg_sentiment_score,rating_normalized,rating_sentiment_gap,product_count,abusive_label,conditions_met_count,label_name
0,(주)경일종합식품,4.950000,0.223607,0.000000,468.700000,0.000000,0.050000,0.421053,20,20.0,...,19.0,0.950000,0.050000,0.950000,0.987500,0.037500,1,1,4,악성
1,(주)그린바드,4.750000,0.910465,0.050000,66.400000,0.125000,0.000000,0.611111,20,20.0,...,12.0,0.600000,0.000000,1.000000,0.937500,0.062500,1,0,1,정상
2,(주)다노트정보,5.000000,0.000000,0.000000,414.421053,0.000000,0.157895,0.473684,19,19.0,...,17.0,0.894737,0.157895,0.842105,1.000000,0.157895,1,1,5,악성
3,(주)리빙스토리,4.333333,1.278019,0.142857,145.571429,0.142857,0.047619,0.500000,21,21.0,...,2.0,0.095238,0.047619,0.952381,0.833333,0.119048,1,0,0,정상
4,(주)맥스올(MAXALL),4.700000,0.732695,0.050000,283.400000,0.083333,0.100000,0.312500,20,20.0,...,16.0,0.800000,0.100000,0.900000,0.925000,0.025000,1,0,2,정상


## 📈 2. 인터랙티브 라벨 분포

In [ ]:
# 라벨 분포 계산
label_counts = df['label_name'].value_counts()

# 파이 차트
fig = go.Figure(data=[go.Pie(
    labels=label_counts.index,
    values=label_counts.values,
    hole=0.3,
    marker=dict(colors=['#2ecc71', '#e74c3c']),
    textinfo='label+percent+value',
    hovertemplate='<b>%{label}</b><br>' +
                  '판매자 수: %{value}명<br>' +
                  '비율: %{percent}<br>' +
                  '<extra></extra>'
)])

fig.update_layout(
    title='판매자 라벨 분포',
    height=500,
    showlegend=True
)

fig.show()

## 🎨 3. Feature 분포 비교 (Interactive Box Plot)

In [5]:
# 주요 Feature 선택
key_features = [
    'review_density', 'question_density', 'avg_rating',
    'negative_sentiment_ratio', 'rating_sentiment_gap', 'question_review_ratio'
]

feature_names_kr = {
    'review_density': '리뷰 밀도',
    'question_density': '문의 밀도',
    'avg_rating': '평균 평점',
    'negative_sentiment_ratio': '부정 감성 비율',
    'rating_sentiment_gap': '평점-감성 괴리도',
    'question_review_ratio': '문의/리뷰 비율'
}

# Subplot 생성
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[feature_names_kr[f] for f in key_features],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

for idx, feature in enumerate(key_features):
    row = idx // 3 + 1
    col = idx % 3 + 1
    
    # 정상
    normal_data = df[df['label_name'] == '정상'][feature]
    fig.add_trace(
        go.Box(
            y=normal_data,
            name='정상',
            marker_color='#2ecc71',
            boxmean='sd',
            hovertemplate='<b>정상 판매자</b><br>' +
                          feature_names_kr[feature] + ': %{y:.3f}<br>' +
                          '<extra></extra>',
            showlegend=(idx == 0)
        ),
        row=row, col=col
    )
    
    # 악성
    abusive_data = df[df['label_name'] == '악성'][feature]
    fig.add_trace(
        go.Box(
            y=abusive_data,
            name='악성',
            marker_color='#e74c3c',
            boxmean='sd',
            hovertemplate='<b>악성 판매자</b><br>' +
                          feature_names_kr[feature] + ': %{y:.3f}<br>' +
                          '<extra></extra>',
            showlegend=(idx == 0)
        ),
        row=row, col=col
    )

fig.update_layout(
    title_text='정상 vs 악성 Feature 비교 (클릭하여 확대/축소 가능)',
    height=800,
    showlegend=True
)

fig.show()

## 🔥 4. Feature Importance (Interactive Bar Chart)

In [ ]:
# Top 15 Feature
top_features = feature_importance.head(15).sort_values('importance')

fig = go.Figure(data=[go.Bar(
    x=top_features['importance'],
    y=top_features['feature'],
    orientation='h',
    marker=dict(
        color=top_features['importance'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Importance")
    ),
    text=top_features['importance'].round(1),
    textposition='outside',
    hovertemplate='<b>%{y}</b><br>' +
                  'Importance: %{x:.1f}<br>' +
                  '순위: %{customdata}위<br>' +
                  '<extra></extra>',
    customdata=list(range(15, 0, -1))
)])

fig.update_layout(
    title='Feature Importance Top 15',
    xaxis_title='Importance (Gain)',
    yaxis_title='',
    height=600,
    showlegend=False
)

fig.show()

## 🎯 5. 산점도 - 주요 Feature 관계

In [7]:
# 부정 감성 비율 vs 평점-감성 괴리도
fig = px.scatter(
    df,
    x='negative_sentiment_ratio',
    y='rating_sentiment_gap',
    color='label_name',
    size='question_count',
    hover_data={
        'vendor_name': True,
        'negative_sentiment_ratio': ':.3f',
        'rating_sentiment_gap': ':.3f',
        'avg_rating': ':.2f',
        'question_count': True,
        'label_name': True
    },
    color_discrete_map={'정상': '#2ecc71', '악성': '#e74c3c'},
    title='부정 감성 비율 vs 평점-감성 괴리도 (점 크기 = 문의 개수)',
    labels={
        'negative_sentiment_ratio': '부정 감성 비율',
        'rating_sentiment_gap': '평점-감성 괴리도',
        'label_name': '판매자 유형',
        'vendor_name': '판매자명'
    },
    height=600
)

fig.update_traces(marker=dict(line=dict(width=1, color='white')))
fig.show()

## 📊 6. 3D 산점도 - 세 가지 Feature 동시 비교

In [8]:
# 3D 산점도
fig = px.scatter_3d(
    df,
    x='negative_sentiment_ratio',
    y='rating_sentiment_gap',
    z='question_review_ratio',
    color='label_name',
    size='conditions_met_count',
    hover_data={
        'vendor_name': True,
        'negative_sentiment_ratio': ':.3f',
        'rating_sentiment_gap': ':.3f',
        'question_review_ratio': ':.3f',
        'conditions_met_count': True,
        'label_name': True
    },
    color_discrete_map={'정상': '#2ecc71', '악성': '#e74c3c'},
    title='3D 시각화 - 주요 Feature 관계 (마우스로 회전 가능)',
    labels={
        'negative_sentiment_ratio': '부정 감성 비율',
        'rating_sentiment_gap': '평점-감성 괴리도',
        'question_review_ratio': '문의/리뷰 비율',
        'label_name': '판매자 유형',
        'vendor_name': '판매자명'
    },
    height=700
)

fig.update_traces(marker=dict(line=dict(width=0.5, color='white')))
fig.show()

## 🎪 7. 애니메이션 - 조건 충족 개수별 변화

In [9]:
# 조건별 그룹화
df_animation = df.copy()
df_animation['조건_그룹'] = df_animation['conditions_met_count'].apply(
    lambda x: f'{x}개 충족' if x <= 5 else '6개 충족'
)

# 애니메이션 산점도
fig = px.scatter(
    df_animation,
    x='negative_sentiment_ratio',
    y='rating_sentiment_gap',
    animation_frame='조건_그룹',
    color='label_name',
    size='question_count',
    hover_data={
        'vendor_name': True,
        'conditions_met_count': True,
        'avg_rating': ':.2f'
    },
    color_discrete_map={'정상': '#2ecc71', '악성': '#e74c3c'},
    title='조건 충족 개수별 판매자 분포 (재생 버튼 클릭)',
    labels={
        'negative_sentiment_ratio': '부정 감성 비율',
        'rating_sentiment_gap': '평점-감성 괴리도',
        'label_name': '판매자 유형'
    },
    range_x=[0, df['negative_sentiment_ratio'].max() * 1.1],
    range_y=[0, df['rating_sentiment_gap'].max() * 1.1],
    height=600
)

fig.update_layout(transition={'duration': 500})
fig.show()

## 🔍 8. 상위/하위 판매자 인터랙티브 테이블

In [10]:
# 가장 의심스러운 판매자
top_suspicious = df.nlargest(10, 'conditions_met_count')[[
    'vendor_name', 'conditions_met_count', 'label_name',
    'negative_sentiment_ratio', 'rating_sentiment_gap', 'avg_rating'
]].round(3)

fig = go.Figure(data=[go.Table(
    header=dict(
        values=['판매자명', '조건 충족', '라벨', '부정 감성', '평점 괴리', '평균 평점'],
        fill_color='#e74c3c',
        font=dict(color='white', size=12),
        align='center'
    ),
    cells=dict(
        values=[top_suspicious[col] for col in top_suspicious.columns],
        fill_color=[['#ffe6e6' if i % 2 == 0 else 'white' for i in range(len(top_suspicious))]],
        align='left'
    )
)])

fig.update_layout(
    title='가장 의심스러운 판매자 Top 10',
    height=400
)

fig.show()

## 📈 9. 모델 성능 - ROC Curve

In [11]:
from sklearn.metrics import roc_curve, roc_auc_score

# ROC 곡선 계산
fpr, tpr, thresholds = roc_curve(predictions['actual'], predictions['probability'])
auc = roc_auc_score(predictions['actual'], predictions['probability'])

# 인터랙티브 ROC Curve
fig = go.Figure()

# ROC 곡선
fig.add_trace(go.Scatter(
    x=fpr, y=tpr,
    mode='lines',
    name=f'ROC Curve (AUC = {auc:.4f})',
    line=dict(color='darkorange', width=3),
    hovertemplate='FPR: %{x:.3f}<br>TPR: %{y:.3f}<br>Threshold: %{customdata:.3f}<extra></extra>',
    customdata=thresholds
))

# 대각선 (Random)
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    name='Random (AUC = 0.5)',
    line=dict(color='navy', width=2, dash='dash'),
    hoverinfo='skip'
))

fig.update_layout(
    title=f'ROC Curve (AUC = {auc:.4f})',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    height=600,
    hovermode='closest'
)

fig.show()

## 🎨 10. Confusion Matrix (Interactive Heatmap)

In [12]:
from sklearn.metrics import confusion_matrix

# Confusion Matrix 계산
cm = confusion_matrix(predictions['actual'], predictions['predicted'])

# 인터랙티브 히트맵
fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=['예측: 정상', '예측: 악성'],
    y=['실제: 정상', '실제: 악성'],
    colorscale='Blues',
    text=cm,
    texttemplate='%{text}명',
    textfont={"size": 20},
    hovertemplate='%{y}<br>%{x}<br>개수: %{z}명<extra></extra>'
))

fig.update_layout(
    title='Confusion Matrix (정확도: 92.0%)',
    height=500,
    xaxis={'side': 'bottom'}
)

fig.show()

## 🎉 완료!

### 인터랙티브 기능:
- ✅ **마우스 호버**: 상세 정보 표시
- ✅ **줌/패닝**: 클릭 드래그로 확대/축소
- ✅ **범례 클릭**: 데이터 계열 숨기기/보이기
- ✅ **3D 회전**: 마우스로 회전 가능
- ✅ **애니메이션**: 재생 버튼으로 동적 표시
- ✅ **다운로드**: 우측 상단 아이콘으로 이미지 저장